# 🌍 Global Market Attractiveness Analysis using Open Data

This notebook fetches data from the World Bank API to calculate a Market Attractiveness Score for countries based on key economic and digital indicators.

In [ ]:
# Install wbdata if needed (for Google Colab)
!pip install wbdata


In [ ]:
import wbdata
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
# Define indicators and date range
indicators = {
    'NY.GDP.PCAP.CD': 'GDP_per_capita',
    'SP.POP.TOTL': 'Population',
    'IT.NET.USER.ZS': 'Internet_pct',
    'NE.EXP.GNFS.ZS': 'Exports_pct_GDP'
}

start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 12, 31)


In [ ]:
# Fetch data from World Bank API
df = wbdata.get_dataframe(indicators, country='all', data_date=(start_date, end_date), convert_date=False)
df = df.reset_index()
df = df.dropna()
df.head()


## Normalize Data and Calculate Score

In [ ]:
columns_to_normalize = ['GDP_per_capita', 'Population', 'Internet_pct', 'Exports_pct_GDP']

scaler = MinMaxScaler()
df_normalized = df.copy()

df_normalized[['GDP_norm', 'Pop_norm', 'Internet_norm', 'Export_norm']] = scaler.fit_transform(
    df_normalized[columns_to_normalize]
)


In [ ]:
df_normalized['Market_Attractiveness_Score'] = (
    0.3 * df_normalized['GDP_norm'] +
    0.25 * df_normalized['Pop_norm'] +
    0.25 * df_normalized['Internet_norm'] +
    0.2 * df_normalized['Export_norm']
)


In [ ]:
df_ranked = df_normalized[['Country', 'Market_Attractiveness_Score']].sort_values(
    by='Market_Attractiveness_Score', ascending=False
)
df_ranked.head(10)


## Bar Chart: Top 10 Countries

In [ ]:
top10 = df_ranked.head(10)

plt.figure(figsize=(10,6))
plt.barh(top10['Country'][::-1], top10['Market_Attractiveness_Score'][::-1], color='skyblue')
plt.xlabel("Market Attractiveness Score")
plt.title("Top 10 Most Attractive Countries")
plt.tight_layout()
plt.show()


## Global Map

In [ ]:
fig = px.choropleth(
    df_normalized,
    locations="Country",
    locationmode="country names",
    color="Market_Attractiveness_Score",
    hover_name="Country",
    color_continuous_scale="Viridis",
    title="Global Market Attractiveness Score by Country"
)
fig.show()
